In [ ]:
!pip install gradio
!pip install audiomentations -q
!pip install wandb
!pip install pytorch_lightning
!pip install gradio
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.1 MB

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import random
import wandb
import torch.utils
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import os
import csv
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import scipy
import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns
import gradio as gr
from audiomentations import Compose, AddGaussianNoise, Gain, AddBackgroundNoise, PitchShift, Shift, TimeMask, AdjustDuration, Normalize
from google.colab import files
import io
from io import BytesIO
from PIL import Image
import torchvision

In [ ]:
#Our modell cls
class AudioClassifier(pl.LightningModule):
    def __init__(self, model):
        super(AudioClassifier, self).__init__()
        self.model = model
        self.y_true = []
        self.y_pred = []
        self.y_prob = []

    def forward(self, x):
        return self.model(x)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y = y.squeeze()
        y_pred = self(x)
        test_loss = F.cross_entropy(y_pred, y)
        y_proba = F.softmax(y_pred, dim=1)
        y_pred = torch.argmax(self(x), dim=1)
        test_acc = torch.sum(y_pred == y).item() / y.size(0)
        self.y_true.extend(y.cpu().numpy())
        self.y_pred.extend(y_pred.cpu().numpy())
        self.y_prob.extend(y_proba.cpu().numpy())
        self.log('test_loss', test_loss.item(), on_epoch=True)
        self.log('test_acc', test_acc, on_epoch=True)


    def get_results(self):
        return self.y_true,self.y_pred,self.y_prob

In [ ]:
augmentations = Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.4),
            Gain(min_gain_in_db=-12, max_gain_in_db=12, p=0.4),
            Shift(min_shift=1, max_shift=6, shift_unit="seconds", p=0.6)
        ])

In [ ]:
label_mapping = {"abethr1": 0, "abhori1": 1, "abythr1": 2, "afbfly1": 3, "afdfly1": 4, "afecuc1": 5, "affeag1": 6, "afgfly1": 7, "afghor1": 8, "afmdov1": 9, "afpfly1": 10, "afpkin1": 11, "afpwag1": 12, "afrgos1": 13, "afrgrp1": 14, "afrjac1": 15, "afrthr1": 16, "amesun2": 17, "augbuz1": 18, "bagwea1": 19, "barswa": 20, "bawhor2": 21, "bawman1": 22, "bcbeat1": 23, "beasun2": 24, "bkctch1": 25, "bkfruw1": 26, "blacra1": 27, "blacuc1": 28, "blakit1": 29, "blaplo1": 30, "blbpuf2": 31, "blcapa2": 32, "blfbus1": 33, "blhgon1": 34, "blhher1": 35, "blksaw1": 36, "blnmou1": 37, "blnwea1": 38, "bltapa1": 39, "bltbar1": 40, "bltori1": 41, "blwlap1": 42, "brcale1": 43, "brcsta1": 44, "brctch1": 45, "brcwea1": 46, "brican1": 47, "brobab1": 48, "broman1": 49, "brosun1": 50, "brrwhe3": 51, "brtcha1": 52, "brubru1": 53, "brwwar1": 54, "bswdov1": 55, "btweye2": 56, "bubwar2": 57, "butapa1": 58, "cabgre1": 59, "carcha1": 60, "carwoo1": 61, "categr": 62, "ccbeat1": 63, "chespa1": 64, "chewea1": 65, "chibat1": 66, "chtapa3": 67, "chucis1": 68, "cibwar1": 69, "cohmar1": 70, "colsun2": 71, "combul2": 72, "combuz1": 73, "comsan": 74, "crefra2": 75, "crheag1": 76, "crohor1": 77, "darbar1": 78, "darter3": 79, "didcuc1": 80, "dotbar1": 81, "dutdov1": 82, "easmog1": 83, "eaywag1": 84, "edcsun3": 85, "egygoo": 86, "equaka1": 87, "eswdov1": 88, "eubeat1": 89, "fatrav1": 90, "fatwid1": 91, "fislov1": 92, "fotdro5": 93, "gabgos2": 94, "gargan": 95, "gbesta1": 96, "gnbcam2": 97, "gnhsun1": 98, "gobbun1": 99, "gobsta5": 100, "gobwea1": 101, "golher1": 102, "grbcam1": 103, "grccra1": 104, "grecor": 105, "greegr": 106, "grewoo2": 107, "grwpyt1": 108, "gryapa1": 109, "grywrw1": 110, "gybfis1": 111, "gycwar3": 112, "gyhbus1": 113, "gyhkin1": 114, "gyhneg1": 115, "gyhspa1": 116, "gytbar1": 117, "hadibi1": 118, "hamerk1": 119, "hartur1": 120, "helgui": 121, "hipbab1": 122, "hoopoe": 123, "huncis1": 124, "hunsun2": 125, "joygre1": 126, "kerspa2": 127, "klacuc1": 128, "kvbsun1": 129, "laudov1": 130, "lawgol": 131, "lesmaw1": 132, "lessts1": 133, "libeat1": 134, "litegr": 135, "litswi1": 136, "litwea1": 137, "loceag1": 138, "lotcor1": 139, "lotlap1": 140, "luebus1": 141, "mabeat1": 142, "macshr1": 143, "malkin1": 144, "marsto1": 145, "marsun2": 146, "mcptit1": 147, "meypar1": 148, "moccha1": 149, "mouwag1": 150, "ndcsun2": 151, "nobfly1": 152, "norbro1": 153, "norcro1": 154, "norfis1": 155, "norpuf1": 156, "nubwoo1": 157, "pabspa1": 158, "palfly2": 159, "palpri1": 160, "piecro1": 161, "piekin1": 162, "pitwhy": 163, "purgre2": 164, "pygbat1": 165, "quailf1": 166, "ratcis1": 167, "raybar1": 168, "rbsrob1": 169, "rebfir2": 170, "rebhor1": 171, "reboxp1": 172, "reccor": 173, "reccuc1": 174, "reedov1": 175, "refbar2": 176, "refcro1": 177, "reftin1": 178, "refwar2": 179, "rehblu1": 180, "rehwea1": 181, "reisee2": 182, "rerswa1": 183, "rewsta1": 184, "rindov": 185, "rocmar2": 186, "rostur1": 187, "ruegls1": 188, "rufcha2": 189, "sacibi2": 190, "sccsun2": 191, "scrcha1": 192, "scthon1": 193, "shesta1": 194, "sichor1": 195, "sincis1": 196, "slbgre1": 197, "slcbou1": 198, "sltnig1": 199, "sobfly1": 200, "somgre1": 201, "somtit4": 202, "soucit1": 203, "soufis1": 204, "spemou2": 205, "spepig1": 206, "spewea1": 207, "spfbar1": 208, "spfwea1": 209, "spmthr1": 210, "spwlap1": 211, "squher1": 212, "strher": 213, "strsee1": 214, "stusta1": 215, "subbus1": 216, "supsta1": 217, "tacsun1": 218, "tafpri1": 219, "tamdov1": 220, "thrnig1": 221, "trobou1": 222, "varsun2": 223, "vibsta2": 224, "vilwea1": 225, "vimwea1": 226, "walsta1": 227, "wbgbir1": 228, "wbrcha2": 229, "wbswea1": 230, "wfbeat1": 231, "whbcan1": 232, "whbcou1": 233, "whbcro2": 234, "whbtit5": 235, "whbwea1": 236, "whbwhe3": 237, "whcpri2": 238, "whctur2": 239, "wheslf1": 240, "whhsaw1": 241, "whihel1": 242, "whrshr1": 243, "witswa1": 244, "wlwwar": 245, "wookin1": 246, "woosan": 247, "wtbeat1": 248, "yebapa1": 249, "yebbar1": 250, "yebduc1": 251, "yebere1": 252, "yebgre1": 253, "yebsto1": 254, "yeccan1": 255, "yefcan": 256, "yelbis1": 257, "yenspu1": 258, "yertin1": 259, "yesbar1": 260, "yespet1": 261, "yetgre1": 262, "yewgre1": 263}

In [ ]:
# Function that runs on Gradio file upload on Submit click
def visualize_melspec(audio_file):
    file_pth = audio_file.name

    # Load the audio file
    waveform, sample_rate = librosa.load(file_pth, sr=None)

    # Apply augmentations
    waveform = augmentations(waveform, sample_rate=sample_rate)

    # Apply normalization transform
    normalization_transform = Normalize(p=1.0)
    waveform = normalization_transform(waveform, sample_rate=sample_rate)

    # Apply duration transformations do have all Mel Specs as same size
    duration_transform = AdjustDuration(duration_seconds=10, padding_mode="wrap", p=1.0)
    waveform = duration_transform(waveform, sample_rate=sample_rate)

    # Compute the mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=waveform, sr=sample_rate, fmin=800, fmax=12000)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    mel_spectrogram_db = normalize_mel_spectrogram(mel_spectrogram_db)

    # Add batch and channel dimensions
    input_tensor = torch.tensor(mel_spectrogram_db).unsqueeze(0).unsqueeze(0)
    # Reverse label encoding
    index_to_label = {v: k for k, v in label_mapping.items()}

    # Predict on the loaded model from WandbAi
    with torch.no_grad():
        logits = eval_model(input_tensor)

    probabilities = torch.nn.functional.softmax(logits, dim=1)
    # Get most probable predicted class
    predicted_class = torch.argmax(probabilities, dim=1).item()
    # Reverse the label to number mapping to get original bird identifier
    predicted_label = index_to_label.get(predicted_class, 'Unknown')

    # Convert the Mel Spectrogram to a format that is accepted by Gradio Image output
    fig = plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spectrogram_db, x_axis='time', y_axis='mel', sr=sample_rate)
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Mel Spectrogram of Bird')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()

    fig_io = io.BytesIO()
    fig.savefig(fig_io, format='png')
    img = Image.open(fig_io)

    return file_pth, img, predicted_label

In [ ]:
# Normalize the Mel Spectrogram input tensor data
def normalize_mel_spectrogram(mel_spec_tensor):
    min_value = mel_spec_tensor.min()
    max_value = mel_spec_tensor.max()
    normalized_mel_spec = (mel_spec_tensor - min_value) / (max_value - min_value)
    return normalized_mel_spec

In [ ]:
# WandbAi initialization and retrival of the current best modell
run = wandb.init()
model_name = 'model-r4t5mun3:v15'
artifact = run.use_artifact('deepbirding/deepbirding/' + model_name, type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact model-r4t5mun3:v15, 81.82MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.3


In [ ]:
num_classes = 264
eval_model = torchvision.models.resnet34(pretrained=True)
eval_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
eval_model.fc = nn.Linear(eval_model.fc.in_features, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
checkpoint = torch.load(f'/content/artifacts/{model_name}/model.ckpt')

# Modify keys in the state dictionary to exclude model. if it exists
new_state_dict = {}
for key, value in checkpoint['state_dict'].items():
    if key.startswith("model."):
        key = key[6:]  # Remove the "model." prefix, because sometimes the modell retrieved from wandbAi project appends to all keys an additional model. part
    new_state_dict[key] = value

eval_model.load_state_dict(new_state_dict)

eval_model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [35]:
# Create Gradio interface to visualize the results of a chosen audio file
iface = gr.Interface(fn=visualize_melspec, inputs=gr.File(), outputs=[gr.Audio(label="Original Audio"),
        gr.Image(label="Mel Spectrogram of Original Audio"), gr.Textbox(label="Predicted bird")

    ])
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e204e65badb31c99d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://e204e65badb31c99d5.gradio.live
